In [1]:
import email
import imaplib
import re
from datetime import datetime
from io import BytesIO

import camelot
import pandas as pd
from bs4 import BeautifulSoup

C:\Users\cw\AppData\Roaming\Python\Python311\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [2]:
prev = 0


def pretty(label: str, completed: float, total: int, length: int = 30) -> None:
    global prev
    print(
        " " * (prev * 2)
        + f"\r{label} ["
        + "=" * int(completed / total * length)
        + "-" * int((total - completed) / total * length)
        + f"] {int(completed / total * 100)}%",
        end="",
    )
    prev = length


def prettyPrintReplace(label: str) -> None:
    global prev
    label = str(label)
    print(" " * (prev * 2) + f"\r{label}", end="")
    prev = len(label)

In [3]:
# Login credentials
IMAP_HOST = "imap.gmail.com"
EMAIL_USER = "ankushmisal7387@gmail.com"
EMAIL_PASS = "jdax zclo xqxz eacr"

In [4]:
subject_to_search = "UPGRADE MANDI - Hyperpure"

In [ ]:
# Connect and login
mail = imaplib.IMAP4_SSL(IMAP_HOST)
mail.login(EMAIL_USER, EMAIL_PASS)
mail.select("inbox")

dt = "09-Jun-2025"

dateFilter = f'ON "{dt}" '

if True:
    dateFilter = ""

# Search emails by subject
statusRaw, messagesRaw = mail.search(
    None, f'({dateFilter}SUBJECT "{subject_to_search}")'
)

In [10]:
grnPrDF = pd.DataFrame({"Cut/Veg": [], "Date": [], "GRN": [], "PO Amount": []})

status = statusRaw
messages = messagesRaw

if status == "OK":
    for num in messages[0].split():
        typ, data = mail.fetch(num, "(RFC822)")
        msg = email.message_from_bytes(data[0][1])

        processedDOMTree = ""
        cutVeg = {"value": None}

        subject = re.sub("\s+", " ", msg["subject"])
        indexPO = re.search(r"CPCMH\d{2}-PO-\d{7}", subject)

        if indexPO:
            indexPO = indexPO.group()
        else:
            raise Exception("Can't find PO number from mail.")

        if indexPO not in grnPrDF.index:
            grnPrDF.loc[indexPO] = ["", "", 0.0, 0.0]

        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition") or "")

            if part.get_content_maintype() == "multipart":
                continue
            elif content_type == "application/pdf":
                cutVeg = "VEG"
                pdfFileData = part.get_payload(decode=True)
                pdfFileStream = BytesIO(pdfFileData)
                try:
                    pdfParserObject = camelot.read_pdf(pdfFileStream)
                    if len(pdfParserObject[0].df) >= 5:
                        findCut = pdfParserObject[0].df.iloc[5][1]
                        findMatches = re.findall(r".*cut.*", findCut, re.IGNORECASE)
                        if len(findMatches) != 0:
                            cutVeg = "CUT-VEG"
                except:
                    cutVeg = "Error"
                grnPrDF.loc[indexPO, "Cut/Veg"] = cutVeg
            elif content_type == "text/html":
                rawHTMLText = part.get_payload(decode=True).decode()

                processedDOMTree = BeautifulSoup(rawHTMLText, "html.parser")

                amountDate = datetime.strptime(
                    processedDOMTree.find_all("table")[1]
                    .find_all("tr")[3]
                    .find_all("td")[1]
                    .find("span")
                    .text,
                    "%d %b %Y",
                ).strftime("%d-%m-%Y")

                amount = float(
                    processedDOMTree.find_all("table")[1]
                    .find_all("tr")[4]
                    .find_all("td")[1]
                    .find("span")
                    .text
                )

                isGrnAmount = (
                    "UPGRADE MANDI - Hyperpure GRN against PO Number" in subject
                )
                isTotalAmount = "UPGRADE MANDI - Hyperpure PO Number" in subject

                if isGrnAmount:
                    grnPrDF.loc[indexPO, "GRN"] += amount
                if isTotalAmount:
                    grnPrDF.loc[indexPO, "PO Amount"] += amount
                grnPrDF.loc[indexPO, "Date"] = amountDate

        print(
            f"Count: {len(grnPrDF)}",
            f"Date: {amountDate}",
            f"Index PO: {indexPO}",
            f'GRN total: {grnPrDF.iloc[-1]["GRN"]}',
            f'Total: {grnPrDF.iloc[-1]["PO Amount"]}',
            f'Cut: {grnPrDF.iloc[-1]["Cut/Veg"]}',
            sep="\n",
            end="\n\n\n",
        )

Count: 1
Date: 05-12-2024
Index PO: CPCMH25-PO-1269666
GRN total: 0.0
Total: 18806.0
Cut: VEG


Count: 1
Date: 05-12-2024
Index PO: CPCMH25-PO-1269666
GRN total: 18788.5
Total: 18806.0
Cut: VEG


Count: 2
Date: 07-12-2024
Index PO: CPCMH25-PO-1274368
GRN total: 0.0
Total: 3679.5
Cut: CUT-VEG


Count: 2
Date: 07-12-2024
Index PO: CPCMH25-PO-1274368
GRN total: 3679.5
Total: 3679.5
Cut: CUT-VEG


Count: 3
Date: 08-12-2024
Index PO: CPCMH25-PO-1276694
GRN total: 0.0
Total: 12956.5
Cut: CUT-VEG


Count: 3
Date: 08-12-2024
Index PO: CPCMH25-PO-1276694
GRN total: 12899.5
Total: 12956.5
Cut: CUT-VEG


Count: 4
Date: 09-12-2024
Index PO: CPCMH25-PO-1279272
GRN total: 0.0
Total: 9028.0
Cut: VEG


Count: 4
Date: 09-12-2024
Index PO: CPCMH25-PO-1279272
GRN total: 9014.0
Total: 9028.0
Cut: VEG


Count: 5
Date: 10-12-2024
Index PO: CPCMH25-PO-1281280
GRN total: 0.0
Total: 55274.2
Cut: CUT-VEG


Count: 5
Date: 10-12-2024
Index PO: CPCMH25-PO-1281280
GRN total: 52301.7
Total: 55274.2
Cut: CUT-VEG


Co

EOF marker not found


Count: 307
Date: 09-06-2025
Index PO: CPCMH26-PO-1785951
GRN total: 0.0
Total: 91994.14
Cut: Error


Count: 307
Date: 09-06-2025
Index PO: CPCMH26-PO-1785948
GRN total: 0.0
Total: 91994.14
Cut: Error


Count: 307
Date: 09-06-2025
Index PO: CPCMH26-PO-1785951
GRN total: 88187.64
Total: 91994.14
Cut: VEG


Count: 308
Date: 10-06-2025
Index PO: CPCMH26-PO-1789620
GRN total: 0.0
Total: 14057.0
Cut: CUT-VEG


Count: 309
Date: 10-06-2025
Index PO: CPCMH26-PO-1789612
GRN total: 0.0
Total: 106908.9
Cut: VEG


Count: 310
Date: 10-06-2025
Index PO: CPCMH26-PO-1790202
GRN total: 0.0
Total: 975.0
Cut: VEG


Count: 310
Date: 10-06-2025
Index PO: CPCMH26-PO-1790202
GRN total: 975.0
Total: 975.0
Cut: VEG


Count: 310
Date: 10-06-2025
Index PO: CPCMH26-PO-1789620
GRN total: 975.0
Total: 975.0
Cut: VEG


Count: 310
Date: 10-06-2025
Index PO: CPCMH26-PO-1789612
GRN total: 975.0
Total: 975.0
Cut: VEG


Count: 311
Date: 11-06-2025
Index PO: CPCMH26-PO-1793082
GRN total: 0.0
Total: 11818.0
Cut: CUT-VEG


C

EOF marker not found


Count: 328
Date: 14-06-2025
Index PO: CPCMH26-PO-1804203
GRN total: 0.0
Total: 114475.0
Cut: VEG


Count: 328
Date: 14-06-2025
Index PO: CPCMH26-PO-1804198
GRN total: 101384.0
Total: 114475.0
Cut: VEG


Count: 329
Date: 14-06-2025
Index PO: CPCMH26-PO-1806887
GRN total: 0.0
Total: 19323.0
Cut: VEG


Count: 329
Date: 14-06-2025
Index PO: CPCMH26-PO-1806887
GRN total: 19323.0
Total: 19323.0
Cut: VEG


Count: 330
Date: 15-06-2025
Index PO: CPCMH26-PO-1807644
GRN total: 0.0
Total: 10191.0
Cut: CUT-VEG


Count: 331
Date: 15-06-2025
Index PO: CPCMH26-PO-1807636
GRN total: 0.0
Total: 120123.0
Cut: VEG


Count: 332
Date: 16-06-2025
Index PO: CPCMH26-PO-1808108
GRN total: 0.0
Total: 7800.0
Cut: VEG


Count: 332
Date: 15-06-2025
Index PO: CPCMH26-PO-1807644
GRN total: 0.0
Total: 7800.0
Cut: VEG


Count: 332
Date: 15-06-2025
Index PO: CPCMH26-PO-1807644
GRN total: 0.0
Total: 7800.0
Cut: VEG


Count: 332
Date: 15-06-2025
Index PO: CPCMH26-PO-1807636
GRN total: 0.0
Total: 7800.0
Cut: VEG


Count: 3

...

Count: 418
Date: 10-07-2025
Index PO: CPCMH26-PO-1900186
GRN total: 0.0
Total: 107033.91
Cut: VEG




EOF marker not found


Count: 418
Date: 10-07-2025
Index PO: CPCMH26-PO-1900179
GRN total: 86331.47
Total: 107033.91
Cut: Error


Count: 419
Date: 11-07-2025
Index PO: CPCMH26-PO-1904138
GRN total: 0.0
Total: 46976.0
Cut: CUT-VEG


Count: 420
Date: 11-07-2025
Index PO: CPCMH26-PO-1904132
GRN total: 0.0
Total: 127986.2
Cut: VEG


Count: 420
Date: 11-07-2025
Index PO: CPCMH26-PO-1904138
GRN total: 0.0
Total: 127986.2
Cut: VEG


Count: 420
Date: 11-07-2025
Index PO: CPCMH26-PO-1904132
GRN total: 90114.7
Total: 127986.2
Cut: VEG


Count: 421
Date: 12-07-2025
Index PO: CPCMH26-PO-1908398
GRN total: 0.0
Total: 25300.0
Cut: VEG


Count: 422
Date: 12-07-2025
Index PO: CPCMH26-PO-1908056
GRN total: 0.0
Total: 117456.2
Cut: VEG


Count: 422
Date: 12-07-2025
Index PO: CPCMH26-PO-1908056
GRN total: 102455.6
Total: 117456.2
Cut: VEG


Count: 422
Date: 12-07-2025
Index PO: CPCMH26-PO-1908398
GRN total: 102455.6
Total: 117456.2
Cut: VEG


Count: 423
Date: 13-07-2025
Index PO: CPCMH26-PO-1911830
GRN total: 0.0
Total: 24500.

EOF marker not found


Count: 447
Date: 24-07-2025
Index PO: CPCMH26-PO-1953988
GRN total: 0.0
Total: 9602.0
Cut: Error


Count: 447
Date: 24-07-2025
Index PO: CPCMH26-PO-1953988
GRN total: 8926.0
Total: 9602.0
Cut: CUT-VEG


Count: 448
Date: 24-07-2025
Index PO: CPCMH26-PO-1953978
GRN total: 0.0
Total: 41277.0
Cut: VEG


Count: 448
Date: 24-07-2025
Index PO: CPCMH26-PO-1953978
GRN total: 34894.5
Total: 41277.0
Cut: VEG


Count: 448
Date: 24-07-2025
Index PO: CPCMH26-PO-1953978
GRN total: 35564.5
Total: 41277.0
Cut: VEG


Count: 449
Date: 25-07-2025
Index PO: CPCMH26-PO-1958182
GRN total: 0.0
Total: 10571.0
Cut: CUT-VEG


Count: 450
Date: 25-07-2025
Index PO: CPCMH26-PO-1958173
GRN total: 0.0
Total: 49206.5
Cut: VEG


Count: 450
Date: 25-07-2025
Index PO: CPCMH26-PO-1958182
GRN total: 0.0
Total: 49206.5
Cut: VEG


Count: 450
Date: 25-07-2025
Index PO: CPCMH26-PO-1958173
GRN total: 38953.0
Total: 49206.5
Cut: VEG


Count: 451
Date: 26-07-2025
Index PO: CPCMH26-PO-1962185
GRN total: 0.0
Total: 11558.0
Cut: CUT

In [11]:
grnPrDF

,Cut/Veg,Date,GRN,PO Amount
CPCMH25-PO-1269666,VEG,05-12-2024,18788.5,18806.0
CPCMH25-PO-1274368,CUT-VEG,07-12-2024,3679.5,3679.5
CPCMH25-PO-1276694,CUT-VEG,08-12-2024,12899.5,12956.5
CPCMH25-PO-1279272,VEG,09-12-2024,9014.0,9028.0
CPCMH25-PO-1281280,CUT-VEG,10-12-2024,52301.7,55274.2
...,...,...,...,...
CPCMH26-PO-2053142,VEG,18-08-2025,27242.0,30793.5
CPCMH26-PO-2057400,CUT-VEG,19-08-2025,22594.0,24013.0
CPCMH26-PO-2057391,VEG,19-08-2025,33254.0,46451.0
CPCMH26-PO-2061476,CUT-VEG,20-08-2025,20412.0,22804.0


In [12]:
grnPrDF["PR"] = grnPrDF["PO Amount"] - grnPrDF["GRN"]

In [13]:
grnPrDF

,Cut/Veg,Date,GRN,PO Amount,PR
CPCMH25-PO-1269666,VEG,05-12-2024,18788.5,18806.0,17.5
CPCMH25-PO-1274368,CUT-VEG,07-12-2024,3679.5,3679.5,0.0
CPCMH25-PO-1276694,CUT-VEG,08-12-2024,12899.5,12956.5,57.0
CPCMH25-PO-1279272,VEG,09-12-2024,9014.0,9028.0,14.0
CPCMH25-PO-1281280,CUT-VEG,10-12-2024,52301.7,55274.2,2972.5
...,...,...,...,...,...
CPCMH26-PO-2053142,VEG,18-08-2025,27242.0,30793.5,3551.5
CPCMH26-PO-2057400,CUT-VEG,19-08-2025,22594.0,24013.0,1419.0
CPCMH26-PO-2057391,VEG,19-08-2025,33254.0,46451.0,13197.0
CPCMH26-PO-2061476,CUT-VEG,20-08-2025,20412.0,22804.0,2392.0


In [14]:
toSave = grnPrDF[["Cut/Veg", "Date", "GRN", "PR", "PO Amount"]]

In [ ]:
mail.logout()

('BYE', [b'LOGOUT Requested'])

In [15]:
toSave.index.name = "PO Number"

In [16]:
toSave.to_csv("GRN PR (Blinkit).csv")